In [ ]:
# Installs
!conda install -c conda-forge geopandas --yes #GeoPandas library to create dataframes with geo-information
!conda install -c conda-forge geopy --yes #Geolocation library
!conda install -c conda-forge geocoder --yes #Geolocation library
!conda install -c conda-forge folium=0.5.0 --yes #Mapping library
!conda install -c conda-forge owslib --yes #Library to read WFS data

In [11]:
# Imports
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopandas as gpd # librabry for GeoPandas

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from requests import Request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins

from owslib.wfs import WebFeatureService #Library for reading WFS data

print('Libraries imported.')

Libraries imported.


### Download Dutch area geolocation and demographic data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

Geolocation data sets for the four digits postal code can amongst others be found at PDOK (Public Services On the Map). PDOK is a platform for accessing geo data sets from Dutch governments. This is current and reliable data for both the public and private sectors. PDOK makes digital geo-information available as data services and files. The PDOK services are based on open data and are therefore freely available to everyone.

PDOK publishes datasets with Dutch postal codes geolocation, together with Dutch demographic data from the Dutch Central Bureau of Statistics (CBS).

See https://www.pdok.nl/-/publicatie-cbs-gegevens-naar-postcode

In [9]:
# URL for WFS backend
url = "https://geodata.nationaalgeoregister.nl/cbspostcode4/wfs"

# Initialize
wfs = WebFeatureService(url=url)

# Service provider 
print('Dataset identification title:', wfs.identification.title, '\n')

# Get WFS version
print('Dataset WFS version:', wfs.version, '\n')

# Available methods
print('Available methods:',[operation.name for operation in wfs.operations], '\n')

# Available data layers
print('Available data layers:',list(wfs.contents), '\n')

# Print all metadata of all layers
print('Metadata:')
for layer, meta in wfs.items():
    print(meta.__dict__, '\n')

Dataset identification title: CBS Postcodes4 

Dataset WFS version: 1.0.0 

Available methods: ['GetCapabilities', 'DescribeFeatureType', 'GetFeature'] 

Available data layers: ['cbspostcode4:postcode42015', 'cbspostcode4:postcode42017', 'cbspostcode4:postcode42016'] 

Metadata:
{'abstract': 'Gegevens per numeriek deel van de postcode voor het jaar 2015', 'metadataUrls': [], 'id': 'cbspostcode4:postcode42015', 'verbOptions': ['{http://www.opengis.net/wfs}Query'], 'keywords': ['CBS, postcode, inwoner, man, vrouw, leeftijd, herkomst, allochtoon, inkomen, ww, bijstand, woning, huur, voorziening, nabijheid, dichtheid'], 'title': 'Postcode 2015, numeriek deel', 'boundingBox': (2.938747595140137, 50.56846156575199, 7.578926097286145, 53.62701798435816, urn:ogc:def:crs:EPSG::28992), 'styles': None, 'timepositions': None, 'auth': <Authentication shared=False username=None password=None cert=None verify=True>, 'boundingBoxWGS84': (3.313578145224139, 47.975220611697736, 3.313639050016488, 47.975

In [12]:
# Get data from WFS

# Fetch the last available year, which is 2017. See available layers in output above
layer = 'cbspostcode4:postcode42017'

# Specify the parameters for fetching the data
params = dict(service='WFS', version=wfs.version, request='GetFeature',
      typeName=layer, outputFormat='json')

# Parse the URL with parameters
q = Request('GET', url, params=params).prepare().url

# Read data from URL
pc4geo = gpd.read_file(q)

#Explore first lines
pc4geo.head()

,id,postcode,aantal_inwoners,aantal_mannen,aantal_vrouwen,aantal_inwoners_0_tot_15_jaar,aantal_inwoners_15_tot_25_jaar,aantal_inwoners_25_tot_45_jaar,aantal_inwoners_45_tot_65_jaar,aantal_inwoners_65_jaar_en_ouder,aantal_part_huishoudens,gemiddelde_huishoudensgrootte,aantal_eenpersoonshuishoudens,aantal_meerpersoonshuishoudens_zonder_kind,aantal_eenouderhuishoudens,aantal_tweeouderhuishoudens,aantal_geboorten,percentage_autochtonen,percentage_westerse_allochtonen,percentage_niet_westerse_allochtonen,aantal_woningen,aantal_woningen_bouwjaar_voor_1945,aantal_woningen_bouwjaar_45_tot_65,aantal_woningen_bouwjaar_65_tot_75,aantal_woningen_bouwjaar_75_tot_85,aantal_woningen_bouwjaar_85_tot_95,aantal_woningen_bouwjaar_95_tot_05,aantal_woningen_bouwjaar_05_tot_15,aantal_woningen_bouwjaar_15_en_later,aantal_meergezins_woningen,gemiddeld_gasverbruik_woning,gemiddeld_elektricteitsverbruik_woning,aantal_personen_met_uitkering_onder_aowlft,omgevingsadressendichtheid,stedelijkheid,geometry
0,postcode42017.1,1011,9645,4990,4655,800,1195,3195,2920,1540,6380,1.5,4175,1305,345,500,75,60,30,10,6110,3605,115,115,800,390,670,370,45,5915,900,2060,825,6907,1,"(POLYGON ((122246.232 487910.177, 122259.064 4..."
1,postcode42017.2,1012,8240,4480,3760,465,1195,4195,1730,650,5960,1.4,4245,1175,190,260,50,50,30,10,5695,4725,20,50,85,490,225,105,-99997,5380,1090,2150,480,8410,1,"(POLYGON ((121995.0292 488243.2021, 121999.664..."
2,postcode42017.3,1013,21080,10550,10525,2640,1860,7685,6115,2775,12580,1.7,7525,2430,1020,1510,280,60,20,20,12825,6930,75,115,1625,1475,445,2005,160,12670,920,1990,2720,6224,1,"(POLYGON ((120506.2192 489494.5513, 120495.812..."
3,postcode42017.4,1014,645,375,270,65,45,400,115,20,330,1.7,170,105,10,45,15,60,20,20,270,20,5,5,5,20,185,-99997,25,225,1000,2160,80,2645,1,"(POLYGON ((120390.1126 489860.7457, 120387.401..."
4,postcode42017.5,1015,14810,7545,7265,1410,1465,5235,4365,2340,9740,1.5,6340,1925,535,860,125,60,30,10,9865,7275,100,235,755,1075,340,80,-99997,9385,1050,1960,1395,10975,1,"(POLYGON ((120665.6423 488535.5, 120668.7853 4..."


This data has area bounderies geolocation data, but not the center long/lat per postal code. Found this dataset for that, which gives long/lat for all four digit postcodes.
https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [ ]:
#4PP LatLong in CSV dataset for Netherlands areas in LatLong geolocation
!wget -q -O '4pp.csv' https://git.tuxm.nl/tuxmachine/postcodes/src/master/4pp.csv
print('Data downloaded!')

In [ ]:
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

In [ ]:
neighlatlong.shape

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [ ]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

In [ ]:
neighlatlong.shape

### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [ ]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

In [ ]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

In [ ]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

In [ ]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

In [ ]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

In [ ]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

In [ ]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

In [ ]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

In [ ]:
#Cast column postcode to INT instead of OBJECT as geolocation dataset has INT
postcode.postcode = postcode.postcode.astype('int64')

In [ ]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

In [ ]:
#Drop unneeded columns
postcode.drop(['id', 'alternatieve_schrijfwijzen', 'soort'], axis=1, inplace = True)
postcode.head()

In [ ]:
postcode.shape

### Show neighbourhoods on map for the cities of Amsterdam, Rotterdam, The Hague and Utrecht

In [ ]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
print("Utrecht", Utrecht_coords)
g = geocoder.arcgis('Amsterdam, Netherlands')
amsterdam_coords = g.latlng
print("Amsterdam", amsterdam_coords)
g = geocoder.arcgis('Rotterdam, Netherlands')
rotterdam_coords = g.latlng
print("Rotterdam", rotterdam_coords)
g = geocoder.arcgis('Den Haag, Netherlands')
denhaag_coords = g.latlng
print("The Hague", denhaag_coords)

In [ ]:
#Extract only woonplaats is Utrecht
utrecht = postcode[postcode.woonplaats.str.contains('utrecht',case=False)]
utrecht.head()

In [ ]:
#Extract only woonplaats is Amsterdam
amsterdam = postcode[postcode.woonplaats.str.contains('amsterdam',case=False)]
amsterdam.head()

In [ ]:
#Extract only woonplaats is The Hague
denhaag = postcode[postcode.woonplaats.str.contains('den haag',case=False)]
denhaag.head()

In [ ]:
#Extract only woonplaats is Rotterdam
rotterdam = postcode[postcode.woonplaats.str.contains('rotterdam',case=False)]
rotterdam.head()

In [ ]:
#Create map of Utrecht with neighbourhoods marked
map_utrecht = folium.Map(location=[Utrecht_coords[0], Utrecht_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(utrecht['latitude'], utrecht['longitude'], utrecht['wijk'], utrecht['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_utrecht)  
    
map_utrecht

In [ ]:
#Create map of Amsterdam with neighbourhoods marked
map_amsterdam = folium.Map(location=[amsterdam_coords[0], amsterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(amsterdam['latitude'], amsterdam['longitude'], amsterdam['wijk'], amsterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

In [ ]:
#Create map of Den Haag with neighbourhoods marked
map_denhaag = folium.Map(location=[denhaag_coords[0], denhaag_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(denhaag['latitude'], denhaag['longitude'], denhaag['wijk'], denhaag['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denhaag)  
    
map_denhaag

In [ ]:
#Create map of Rotterdam with neighbourhoods marked
map_rotterdam = folium.Map(location=[rotterdam_coords[0], rotterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(rotterdam['latitude'], rotterdam['longitude'], rotterdam['wijk'], rotterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rotterdam)  
    
map_rotterdam